# Thesis Project - Google Colab Setup

This notebook sets up the Thesis project in Google Colab.

## For Private Repositories

If your repository is **private**, you have two options:

1. **Use GitHub Personal Access Token** (see Cell 1A below) - Recommended
2. **Upload files manually** (see Cell 2 below)


In [ ]:
# Setup: Clone repository and install dependencies
# IMPORTANT: If your repository is private, you'll need to use a Personal Access Token
# Or use the manual upload method in the next cell

REPO_URL = "https://github.com/baswingen/Thesis.git"

import os
import sys
import shutil
import subprocess

# Clone repository (if not already cloned)
if not os.path.exists('thesis_project'):
    # Remove existing directories if they exist
    for dir_name in ['Thesis', 'thesis_project']:
        if os.path.exists(dir_name):
            shutil.rmtree(dir_name)
    
    # Clone directly to thesis_project
    print(f"Cloning repository from {REPO_URL}...")
    print("⚠️  If this fails, your repository might be private. See alternative method below.")
    
    try:
        result = subprocess.run(
            ['git', 'clone', REPO_URL, 'thesis_project'],
            capture_output=True,
            text=True,
            timeout=60
        )
        
        if result.returncode != 0:
            error_msg = result.stderr if result.stderr else result.stdout
            print(f"\n❌ Git clone failed!")
            print(f"Error: {error_msg}")
            print("\n💡 Solutions:")
            print("   1. If repository is private: Use GitHub Personal Access Token")
            print("   2. Upload files manually: Use the code in the next cell")
            print("   3. Check if repository URL is correct")
            raise RuntimeError(f"Failed to clone repository")
        
        if not os.path.exists('thesis_project'):
            raise RuntimeError("Repository clone failed - directory not found")
        
        print("✅ Repository cloned successfully")
    except subprocess.TimeoutExpired:
        raise RuntimeError("Git clone timed out. Please check your internet connection.")
    except RuntimeError:
        raise  # Re-raise our custom errors
    except Exception as e:
        raise RuntimeError(f"Error cloning repository: {str(e)}")

# Find the project root (handle subdirectories)
def find_project_root(start_path):
    """Find the directory containing requirements.txt and setup.py"""
    current = os.path.abspath(start_path)
    
    # Check common subdirectory patterns
    possible_paths = [
        current,
        os.path.join(current, 'python', 'Thesis'),
        os.path.join(current, 'Thesis'),
    ]
    
    # Also search one level deep
    for root, dirs, files in os.walk(current):
        if 'requirements.txt' in files and 'setup.py' in files:
            return root
        # Don't search too deep
        depth = root[len(current):].count(os.sep)
        if depth > 2:
            break
    
    # Check the possible paths
    for path in possible_paths:
        if os.path.exists(path) and os.path.exists(os.path.join(path, 'requirements.txt')):
            return path
    
    return None

# Find the actual project root
project_root = find_project_root('thesis_project')

if project_root:
    print(f"📁 Found project at: {project_root}")
    
    # Install dependencies (fix NumPy compatibility issue)
    req_file = os.path.join(project_root, 'requirements.txt')
    if os.path.exists(req_file):
        import subprocess
        # First, install/upgrade NumPy to compatible version (fixes TensorFlow compatibility)
        print("Installing NumPy first (for TensorFlow compatibility)...")
        subprocess.run(['pip', 'install', '--upgrade', '--force-reinstall', 'numpy<2.0.0,>=1.24.0'], check=False)
        
        # Then install all other dependencies
        print("Installing other dependencies...")
        subprocess.run(['pip', 'install', '-q', '-r', req_file], check=False)
        
        # Reinstall NumPy again to ensure compatibility (sometimes TensorFlow changes it)
        subprocess.run(['pip', 'install', '--upgrade', '--force-reinstall', 'numpy<2.0.0,>=1.24.0'], check=False)
        
        print("✅ Dependencies installed")
    else:
        print("⚠️  requirements.txt not found")
    
    # Install project in editable mode
    if os.path.exists(os.path.join(project_root, 'setup.py')):
        import subprocess
        subprocess.run(['pip', 'install', '-q', '-e', project_root], check=False)
        print("✅ Project installed")
    else:
        print("⚠️  setup.py not found")
    
    # Add to path
    if project_root not in sys.path:
        sys.path.insert(0, project_root)
    
    src_path = os.path.join(project_root, 'src')
    if os.path.exists(src_path) and src_path not in sys.path:
        sys.path.insert(0, src_path)
    
    print("✅ Setup complete!")
    print(f"📂 Project root: {project_root}")
else:
    print("⚠️  Could not find project files (requirements.txt, setup.py)")
    print("\n📋 Repository structure:")
    if os.path.exists('thesis_project'):
        for root, dirs, files in os.walk('thesis_project'):
            level = root.replace('thesis_project', '').count(os.sep)
            indent = ' ' * 2 * level
            print(f"{indent}{os.path.basename(root)}/")
            subindent = ' ' * 2 * (level + 1)
            for file in files[:5]:  # Show first 5 files
                print(f"{subindent}{file}")
            if len(files) > 5:
                print(f"{subindent}... and {len(files) - 5} more files")
            if level > 2:  # Don't go too deep
                break


⚠️  requirements.txt not found. Make sure the repository was cloned correctly.


## Fix NumPy Compatibility (if you get errors)

If you see "numpy.dtype size changed" errors, run this cell:


In [ ]:
# Fix NumPy compatibility issue (run this if you get numpy.dtype errors)
import subprocess
import sys

print("Fixing NumPy compatibility...")
# Uninstall conflicting packages
subprocess.run([sys.executable, '-m', 'pip', 'uninstall', '-y', 'numpy', 'tensorflow'], check=False)

# Install NumPy first with compatible version
subprocess.run([sys.executable, '-m', 'pip', 'install', '--upgrade', '--force-reinstall', 'numpy==1.26.4'], check=False)

# Then install TensorFlow
subprocess.run([sys.executable, '-m', 'pip', 'install', '--upgrade', '--force-reinstall', 'tensorflow>=2.15.0'], check=False)

# Reinstall NumPy to ensure it's compatible
subprocess.run([sys.executable, '-m', 'pip', 'install', '--upgrade', '--force-reinstall', 'numpy==1.26.4'], check=False)

print("✅ NumPy compatibility fixed!")
print("⚠️  You may need to restart the runtime: Runtime → Restart runtime")


## Alternative: Manual Upload (if git clone fails)

If the repository is private or git clone fails, upload your project files directly:

**Option 1: Upload via Colab UI**
1. Click the folder icon (📁) in the left sidebar
2. Click "Upload to session storage" 
3. Upload your project files (you can zip the project folder and extract it)

**Option 2: Use this code to upload from your computer**


In [3]:
# Alternative: If git clone failed, upload files manually
# Run this cell and use the file picker to upload your project

from google.colab import files
import zipfile
import os

# Upload a zip file of your project
print("Upload your project as a zip file:")
uploaded = files.upload()

# Extract the zip file
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('.')
        # Rename extracted folder to thesis_project if needed
        # Adjust this based on your zip file structure
        if os.path.exists('Thesis'):
            os.rename('Thesis', 'thesis_project')
        print(f"✅ Extracted {filename}")
        
        # Now install dependencies
        if os.path.exists('thesis_project/requirements.txt'):
            %pip install -q -r thesis_project/requirements.txt
            %pip install -q -e thesis_project/
            
            import sys
            sys.path.insert(0, '/content/thesis_project')
            sys.path.insert(0, '/content/thesis_project/src')
            print("✅ Setup complete!")


ModuleNotFoundError: No module named 'google.colab'

## Verify Installation

Check that all frameworks are installed correctly:


In [ ]:
import sys
import numpy as np

# Test NumPy first (most common issue)
try:
    test_array = np.array([1, 2, 3])
    print(f"✅ NumPy: {np.__version__} - Working correctly")
except Exception as e:
    print(f"❌ NumPy error: {e}")
    print("⚠️  Run the 'Fix NumPy Compatibility' cell above, then restart runtime")

# Test PyTorch
try:
    import torch
    print(f"✅ PyTorch: {torch.__version__}")
except Exception as e:
    print(f"❌ PyTorch error: {e}")

# Test TensorFlow (most likely to have NumPy issues)
try:
    import tensorflow as tf
    print(f"✅ TensorFlow: {tf.__version__}")
except Exception as e:
    print(f"❌ TensorFlow error: {e}")
    if "numpy" in str(e).lower() or "dtype" in str(e).lower():
        print("⚠️  This is a NumPy compatibility issue. Run the fix cell above.")

# Test Keras
try:
    import keras
    print(f"✅ Keras: {keras.__version__}")
except Exception as e:
    print(f"❌ Keras error: {e}")

print(f"\n✅ Python: {sys.version.split()[0]}")


✅ Python: 3.12.12
✅ PyTorch: 2.9.1
✅ TensorFlow: 2.20.0
✅ Keras: 3.13.0


/opt/homebrew/Caskroom/miniconda/base/envs/thesis/lib/python3.12/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


## Use Your Project Code

Now you can import and use your project modules:


In [ ]:
# Example: Import from your src package
from src import example

# Use your functions
print(example.hello_world("Colab"))
print(f"2 + 3 = {example.add_numbers(2, 3)}")
